In [19]:
from cayman_office_system import *
from shining_pebbles import get_today

In [33]:
COLUMNS_INSTRUCTION = [
    'Name', 'Account', 'Side', 'Broker', 'Symbol', 'Amount', 
    'Order Type', 'Limit', 'Instructions', 'CFD', 'Rationale'
]

MAPPING_SIDE = {
    'Buy': 'B',
    'Sell': 'S',
    'B': 'B',
    'S': 'S'
}  

default_settings = {
    'Name': None,
    'Account': '715543911501',
    'Broker': 'Samsung',
    'Order Type': 'MKT',
    'Limit': None,
    'Instructions': 'VWAP',
    'CFD': None,
    'Rationale': None
}


class OrderInstruction:
    def __init__(self, date=get_today()):
        self.date = date
        self.file_folder = file_folder['generate']
        self.frame = self.get_frame_of_instruction()
        self.orders = []
        
    def set_account(self, account=None):
        if account is None:
            account = '715543911501'
        self.account = account
        return account
    
    def set_default_sides(self, sides):
        self.default_sides = sides
        return sides

    def set_broker(self, broker=None):
        if broker is None:
            broker = 'Samsung'
        self.broker = broker
        return broker
    
    def set_order_type(self, order_type=None):
        if order_type is None:
            order_type = 'MKT'
        self.order_type = order_type
        return order_type
    
    def set_limit(self, limit=None):
        self.limit = limit
        return limit

    def set_instructions(self, instructions=None):
        if instructions is None:
            instructions = 'VWAP'
        self.instructions = instructions
        return instructions
    
    def set_cfd(self, cfd=None):
        self.cfd = cfd
        return cfd
    
    def set_order(self, order):
        orders = self.orders
        dct = {}
        for key in COLUMNS_INSTRUCTION:
            if key == 'Side':
                dct[key] = MAPPING_SIDE[order['Side']]
            elif key in order:
                dct[key] = order[key]
            else:
                dct[key] = default_settings[key]
        print(dct)
        orders.append(dct)
        self.orders = orders
        return self
    
    def set_orders(self, orders):
        for order in orders:
            self.set_order(order)
        return self   
    
    def get_frame_of_instruction(self):
        df = pd.DataFrame(data=None, columns=COLUMNS_INSTRUCTION)
        self.frame = df
        return df

    def generate(self):
        df = pd.DataFrame(self.orders)
        df['Name'] = df['Symbol'].apply(lambda x: get_ks_market_info()['name'].loc[f'{x} Equity'])
        self.df = df
        return df            

    def export_as_excel_format(self):
        df = self.df
        file_folder = self.file_folder
        file_name = f'genearted_at{get_today("%Y%m%d%H")}-Life Asset trade instruction {self.date.replace("-", "")}.xlsx'
        file_path = os.path.join(file_folder, file_name)
        with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='Sheet1')
            
            # 각 열의 너비를 글자에 맞게 조정
            worksheet = writer.sheets['Sheet1']
            for column in worksheet.columns:
                max_length = 0
                column_letter = column[0].column_letter  # Get the column name
                for cell in column:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = (max_length + 2)
                worksheet.column_dimensions[column_letter].width = adjusted_width

        print(f"DataFrame has been saved to {file_path}")
        return None

    def show_price_info(self):
        df = self.df
        df['Price_last'] = df['Symbol'].apply(lambda x: get_ks_market_info()['price_last'].loc[f'{x} Equity'])
        df['Evaluation_krw'] = df['Price_last'] * df['Amount']
        cols_to_keep = ['Name', 'Symbol', 'Amount', 'Price_last', 'Evaluation_krw']
        df = df[cols_to_keep]
        self.price_info = df
        return df        

In [34]:
get_ks_market_info()

,name_kr,name,market_index,sector,cap,price_last
ticker_bbg,,,,,,
000020 KS Equity,동화약품,DONGWHA PHARM CO LTD,KOSPI Index,Health Care,2133.0,7640.0
000040 KS Equity,KR모터스,KR MOTORS CO LTD,KOSPI Index,Consumer Discretionary,200.0,568.0
000050 KS Equity,경방,KYUNGBANG CO LTD,KOSPI Index,Consumer Discretionary,1625.0,5930.0
000070 KS Equity,삼양홀딩스,SAMYANG HOLDINGS CORP,KOSPI Index,Consumer Staples,5323.0,71600.0
000080 KS Equity,하이트진로,HITE JINRO CO LTD,KOSPI Index,Consumer Staples,14412.0,20550.0
...,...,...,...,...,...,...
950170 KS Equity,JTC,NaN,NaN,NaN,2139.0,4255.0
950190 KS Equity,미투젠,NaN,NaN,NaN,1401.0,10320.0
950200 KS Equity,소마젠,NaN,NaN,NaN,805.0,4190.0


In [35]:
instruction = OrderInstruction()
instruction

In [36]:
instruction.date

'2024-09-10'

In [37]:
frame = instruction.get_frame_of_instruction()
frame 

,Name,Account,Side,Broker,Symbol,Amount,Order Type,Limit,Instructions,CFD,Rationale


In [38]:
order_1 = {
    'Symbol': '007340 KS',
    'Side': 'B',
    'Amount': 2500
}
order_2 = {
    'Symbol': '282690 KS',
    'Side': 'Buy',
    'Amount': 2500
}
order_3_4 = [
    {
        'Symbol': '003030 KS',
        'Side': 'B',
        'Amount': 900
    },
    {
        'Symbol': '271560 KS',
        'Side': 'Buy',
        'Amount': 2200
    }
] 

In [39]:
instruction.set_order(order_1)

{'Name': None, 'Account': '715543911501', 'Side': 'B', 'Broker': 'Samsung', 'Symbol': '007340 KS', 'Amount': 2500, 'Order Type': 'MKT', 'Limit': None, 'Instructions': 'VWAP', 'CFD': None, 'Rationale': None}


In [40]:
instruction.set_order(order_2)

{'Name': None, 'Account': '715543911501', 'Side': 'B', 'Broker': 'Samsung', 'Symbol': '282690 KS', 'Amount': 2500, 'Order Type': 'MKT', 'Limit': None, 'Instructions': 'VWAP', 'CFD': None, 'Rationale': None}


In [41]:
instruction.set_orders(order_3_4)

{'Name': None, 'Account': '715543911501', 'Side': 'B', 'Broker': 'Samsung', 'Symbol': '003030 KS', 'Amount': 900, 'Order Type': 'MKT', 'Limit': None, 'Instructions': 'VWAP', 'CFD': None, 'Rationale': None}
{'Name': None, 'Account': '715543911501', 'Side': 'B', 'Broker': 'Samsung', 'Symbol': '271560 KS', 'Amount': 2200, 'Order Type': 'MKT', 'Limit': None, 'Instructions': 'VWAP', 'CFD': None, 'Rationale': None}


In [42]:
instruction.orders

[{'Name': None,
  'Account': '715543911501',
  'Side': 'B',
  'Broker': 'Samsung',
  'Symbol': '007340 KS',
  'Amount': 2500,
  'Order Type': 'MKT',
  'Limit': None,
  'Instructions': 'VWAP',
  'CFD': None,
  'Rationale': None},
 {'Name': None,
  'Account': '715543911501',
  'Side': 'B',
  'Broker': 'Samsung',
  'Symbol': '282690 KS',
  'Amount': 2500,
  'Order Type': 'MKT',
  'Limit': None,
  'Instructions': 'VWAP',
  'CFD': None,
  'Rationale': None},
 {'Name': None,
  'Account': '715543911501',
  'Side': 'B',
  'Broker': 'Samsung',
  'Symbol': '003030 KS',
  'Amount': 900,
  'Order Type': 'MKT',
  'Limit': None,
  'Instructions': 'VWAP',
  'CFD': None,
  'Rationale': None},
 {'Name': None,
  'Account': '715543911501',
  'Side': 'B',
  'Broker': 'Samsung',
  'Symbol': '271560 KS',
  'Amount': 2200,
  'Order Type': 'MKT',
  'Limit': None,
  'Instructions': 'VWAP',
  'CFD': None,
  'Rationale': None}]

In [43]:
instruction.generate()

,Name,Account,Side,Broker,Symbol,Amount,Order Type,Limit,Instructions,CFD,Rationale
0,DN AUTOMOTIVE CORP,715543911501,B,Samsung,007340 KS,2500,MKT,None,VWAP,None,None
1,DONG AH TIRE & RUBBER CO LTD,715543911501,B,Samsung,282690 KS,2500,MKT,None,VWAP,None,None
2,SEAH STEEL HOLDINGS CORP,715543911501,B,Samsung,003030 KS,900,MKT,None,VWAP,None,None
3,ORION CORP/REPUBLIC OF KOREA,715543911501,B,Samsung,271560 KS,2200,MKT,None,VWAP,None,None


In [44]:
instruction.show_price_info()

,Name,Symbol,Amount,Price_last,Evaluation_krw
0,DN AUTOMOTIVE CORP,007340 KS,2500,94200.0,235500000.0
1,DONG AH TIRE & RUBBER CO LTD,282690 KS,2500,13500.0,33750000.0
2,SEAH STEEL HOLDINGS CORP,003030 KS,900,174700.0,157230000.0
3,ORION CORP/REPUBLIC OF KOREA,271560 KS,2200,88800.0,195360000.0


In [45]:
instruction.export_as_excel_format()

DataFrame has been saved to /Users/juneyoungpark/dev/module-cayman_office_system/cayman_office_system/dataset-generate/genearted_at2024091010-Life Asset trade instruction 20240910.xlsx


In [46]:
new_data = {}
for key in frame.columns:
    if key in order_1:
        new_data[key] = order_1[key]
    else:
        new_data[key] = default_settings[key]
new_data

{'Name': None,
 'Account': '715543911501',
 'Side': 'B',
 'Broker': 'Samsung',
 'Symbol': '007340 KS',
 'Amount': 2500,
 'Order Type': 'MKT',
 'Limit': None,
 'Instructions': 'VWAP',
 'CFD': None,
 'Rationale': None}

In [47]:
new_row = pd.DataFrame([new_data])
new_row

,Name,Account,Side,Broker,Symbol,Amount,Order Type,Limit,Instructions,CFD,Rationale
0,None,715543911501,B,Samsung,007340 KS,2500,MKT,None,VWAP,None,None
